In [1]:
cd mergedDatasets

c:\Users\PSA\Desktop\Development\Learning\SkinCancer\mergedDatasets


# Push the data from Images folder to the respective train and test folders and create their DF's.

In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split


In [3]:
fileName = 'GroundTruth.csv'
df = pd.read_csv(fileName)


In [4]:
df.columns

Index(['image', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC', 'NONE'], dtype='object')

In [5]:
def categoryFinder(singleRow):
    for x in df.columns:
        if singleRow[x]==1:
            return x
    return None


In [6]:
df['CATEGORY']=df.apply(lambda x: categoryFinder(x), axis=1)

In [7]:
trainDf, testDf = train_test_split(df[['image','CATEGORY']], test_size=0.33, random_state=40,shuffle=True)

In [8]:
print(trainDf.shape,testDf.shape)

(6737, 2) (3319, 2)


In [9]:
testDf.CATEGORY.unique()

array(['MEL', 'NV', 'BCC', 'BKL', 'DF', 'VASC', 'AKIEC', 'NONE'],
      dtype=object)

In [10]:
#Setting directory structure and data for model functioning
if not os.path.exists('train'):
    os.makedirs('train')

if not os.path.exists('test'):
    os.makedirs('test')

for _ in trainDf.CATEGORY.unique():
    if not os.path.exists('train/'+_):
        os.makedirs('train/'+_)

for _ in testDf.CATEGORY.unique():
    if not os.path.exists('test/'+_):
        os.makedirs('test/'+_)

In [12]:
#Move the data into the respective FolderStructures.
try:
    for _ in range(trainDf.shape[0]):
        os.rename("images/"+trainDf.iloc[_].image +".jpg", "train/"+trainDf.iloc[_].CATEGORY+"/"+trainDf.iloc[_].image+".jpg")
except Exception as E:
    print('FileNotFoundError')

try:
    for _ in range(testDf.shape[0]):
        os.rename("images/"+testDf.iloc[_].image + ".jpg", "test/" +
                testDf.iloc[_].CATEGORY+"/"+testDf.iloc[_].image+".jpg")
except Exception as E:
    print('FileNotFoundError')


FileNotFoundError
FileNotFoundError


# Using Deeplearning model below.

In [17]:
len(trainDf.CATEGORY.unique())

8

In [13]:
import tensorflow as tf
from tensorflow import keras

In [81]:
trainDs = tf.keras.utils.image_dataset_from_directory(
    os.path.join(os.getcwd(),'train'),
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=24,
    validation_split=None,
    subset=None,
    interpolation="nearest"
)

testDs = tf.keras.utils.image_dataset_from_directory(
    os.path.join(os.getcwd(),'test'),
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=24,
    validation_split=None,
    subset=None,
    interpolation="nearest"
)

Found 6737 files belonging to 8 classes.
Found 3319 files belonging to 8 classes.


In [77]:
6737/32

210.53125

In [78]:
len(trainDs)

211

In [82]:
# Simple Model 
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(256,256,1)))
model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPool2D((3, 3)))
model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPool2D((3, 3)))
model.add(tf.keras.layers.Dense(54, activation='relu'))
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.Dense(8,activation = tf.keras.activations.sigmoid))


In [83]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 254, 254, 64)      640       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 84, 84, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 82, 82, 64)        36928     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 27, 27, 64)       0         
 g2D)                                                            
                                                                 
 dense_15 (Dense)            (None, 27, 27, 54)        3510      
                                                                 
 dropout_11 (Dropout)        (None, 27, 27, 54)      

In [84]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])


In [85]:
history = model.fit(
    trainDs,
    epochs=4,
    verbose="2",
    validation_data=testDs,
    shuffle=True,
    initial_epoch=0,
    use_multiprocessing=True
)

Epoch 1/4
Epoch 2/4
Epoch 3/4
Epoch 4/4


In [91]:
history.history

{'loss': [2.839317560195923,
  1.7928498983383179,
  1.6349717378616333,
  1.509108304977417],
 'accuracy': [0.6266884207725525,
  0.6630547642707825,
  0.6630547642707825,
  0.6630547642707825],
 'val_loss': [1.878008246421814,
  1.7002049684524536,
  1.5547199249267578,
  1.4411108493804932],
 'val_accuracy': [0.6742994785308838,
  0.6742994785308838,
  0.6742994785308838,
  0.6742994785308838]}